# Executive Brief: Support Operations & SLA Optimization
**Prepared By**: Senior Data Analyst

## 1. The Business Problem
Our Support Operations team is facing challenges with inconsistent resolution times and missed SLAs. To address this, we have initiated a comprehensive audit of our ticket data to answer:
1. **Where are we failing?** (Descriptive Analytics)
2. **Why are we failing?** (Statistical & Root Cause Analysis)
3. **How can we fix it?** (Predictive Modeling & Strategic Recommendations)

### Core KPIs Audited
- **SLA Breach Rate**: Target < 10% for Critical Tickets.
- **Resolution Time**: Identifying barriers to speed.
- **Financial Risk**: Quantifying the cost of service failures.

In [ ]:
# 1. Setup & Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, average_precision_score
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

# Settings for cleaner output
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded.")

## 2. Load Data
Ingesting the raw ticket logs for analysis.

In [ ]:
# Load the dataset
try:
    df = pd.read_csv('../data/customer_support_tickets.csv')
    print(f"Dataset loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    try:
        df = pd.read_csv('customer_support_tickets.csv')
        print(f"Dataset loaded successfully. Shape: {df.shape}")
    except FileNotFoundError:
        print("Error: customer_support_tickets.csv not found.")

if 'df' in locals():
    display(df.head())

## 3. SLA Definition & Business Logic (Canonical)
**SINGLE SOURCE OF TRUTH**
Here we define exactly what constitutes a "Breach" and the financial cost associated with it.
Any downstream analysis MUST use `Resolution_Hours` and `Is_SLA_Breach` defined here.

**Logic Rules**:
1. **Ticket Creation**: Imputed (1-5h before first response) due to missing raw log.
2. **Resolution Hours**: `Time Resolved` - `Creation Time`.
3. **SLA Targets**: Critical (4h), High (8h), Normal (24h), Low (72h).

In [ ]:
# --- CANONICAL SLA LOGIC ENGINE ---

# A. Date Conversion
df['Time_Resolved'] = pd.to_datetime(df['Time to Resolution'], errors='coerce')
df['Time_First_Response'] = pd.to_datetime(df['First Response Time'], errors='coerce')

# B. Filter Valid Rows
df_sla = df.dropna(subset=['Time_Resolved', 'Time_First_Response']).copy()

# C. Impute Creation Date (Simulation of Ground Truth)
np.random.seed(42)
random_hours = pd.to_timedelta(np.random.randint(1, 6, size=len(df_sla)), unit='h')
df_sla['Ticket Creation Date'] = df_sla['Time_First_Response'] - random_hours

# D. Calculate Resolution Hours
df_sla['Resolution_Hours'] = (df_sla['Time_Resolved'] - df_sla['Ticket Creation Date']).dt.total_seconds() / 3600
df_sla = df_sla[df_sla['Resolution_Hours'] > 0].copy() # Filter hygiene

# E. Define SLA Targets
def get_sla_target(priority):
    targets = {'Critical': 4, 'High': 8, 'Normal': 24, 'Low': 72}
    return targets.get(priority, 24)

df_sla['SLA_Target_Hours'] = df_sla['Ticket Priority'].apply(get_sla_target)

# F. Determine Breach Status
df_sla['Is_SLA_Breach'] = df_sla['Resolution_Hours'] > df_sla['SLA_Target_Hours']
df_sla['Is_SLA_Breach_Numeric'] = df_sla['Is_SLA_Breach'].astype(int)

# G. Assign Financial Risk (Cost Logic)
def get_breach_cost(row):
    if not row['Is_SLA_Breach']: return 0
    # Cost = Penalty + Churn Risk Estimate
    costs = {'Critical': 500, 'High': 200, 'Normal': 50, 'Low': 10}
    return costs.get(row['Ticket Priority'], 0)

df_sla['Est_Breach_Cost'] = df_sla.apply(get_breach_cost, axis=1)
df_sla['Breach_Cost'] = df_sla['Est_Breach_Cost'] # Alias for modeling

# Extract Hour for Workload Analyis
df_sla['Hour_of_Day'] = df_sla['Ticket Creation Date'].dt.hour

print("✅ SLA Logic & Financial Risk Engine Applied.")
print(f"Analyzable Dataset: {df_sla.shape[0]} tickets.")
display(df_sla[['Ticket Creation Date', 'Resolution_Hours', 'SLA_Target_Hours', 'Is_SLA_Breach', 'Est_Breach_Cost']].head())

## Phase 2A: Descriptive Risk Analytics
**Goal**: Identify "Where are we bleeding?"
Visualizing the operational landscape to pinpoint the bleeding.

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='Ticket Priority', y='Is_SLA_Breach', data=df_sla, order=['Critical', 'High', 'Normal', 'Low'], ci=None, palette='viridis')
plt.title('SLA Breach Rate by Priority')
plt.ylabel('Breach Rate')
plt.axhline(df_sla['Is_SLA_Breach'].mean(), color='red', linestyle='--', label='Overall Average')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df_sla, x='Resolution_Hours', hue='Ticket Priority', bins=50, kde=True, palette='viridis')
plt.title('Distribution of Resolution Time by Priority')
plt.xlim(0, 100) # Zoom in for readability, adjust as needed
plt.show()

## Phase 2B: Diagnostic Root Cause Analysis
**Goal**: Explain "Why is this happening?"
We investigate Volume vs Risk, Channel Friction, and Product Complexity.

In [ ]:
# --- 2B.1 Volume vs Risk Analysis ---
diagnostic_volume = (
    df_sla.groupby(['Ticket Type', 'Ticket Priority'])
      .agg(
          Tickets=('Ticket ID', 'count'),
          Breach_Rate=('Is_SLA_Breach_Numeric', 'mean'),
          Avg_Resolution_Hours=('Resolution_Hours', 'mean'),
          Total_Cost=('Est_Breach_Cost', 'sum')
      )
      .reset_index()
      .sort_values('Total_Cost', ascending=False)
)

print("--- Volume vs Risk: Top Drivers ---")
display(diagnostic_volume.head(10))

# Interpretation: High Cost usually comes from High Volume * High Breach Rate.

In [ ]:
# --- 2B.2 Channel Friction Analysis ---
channel_diagnostics = (
    df_sla.groupby(['Ticket Channel'])
      .agg(
          Tickets=('Ticket ID', 'count'),
          Breach_Rate=('Is_SLA_Breach_Numeric', 'mean'),
          Avg_Resolution=('Resolution_Hours', 'mean'),
          Avg_Cost=('Est_Breach_Cost', 'mean')
      )
      .reset_index()
      .sort_values('Breach_Rate', ascending=False)
)

print("--- Channel Friction Analysis ---")
display(channel_diagnostics)

In [ ]:
# --- 2B.3 Product Complexity Signal ---
product_diagnostics = (
    df_sla.groupby(['Product Purchased'])
      .agg(
          Tickets=('Ticket ID', 'count'),
          Breach_Rate=('Is_SLA_Breach_Numeric', 'mean'),
          Avg_Resolution=('Resolution_Hours', 'mean'),
          Total_Cost=('Est_Breach_Cost', 'sum')
      )
      .query('Tickets >= 10')
      .sort_values('Breach_Rate', ascending=False)
)

print("--- Product Complexity: Top Failure Rates ---")
display(product_diagnostics.head(10))

In [ ]:
# --- 2B.4 Statistical Validation (Chi-Square) ---
print("--- Chi-Square Test: Priority vs Breach ---")
contingency = pd.crosstab(df_sla['Ticket Priority'], df_sla['Is_SLA_Breach'])
chi2, p_value, _, _ = chi2_contingency(contingency)

print(f"Chi-Square Statistic: {chi2:.4f}")
print(f"P-Value: {p_value:.4e}")

if p_value < 0.05:
    print("✅ Result: Statistically Significant. Priority influences Breach Rate.")
else:
    print("❌ Result: Not Significant.")

## Phase 2C: Predictive SLA Breach Modeling
**Goal**: Predict "Which incoming tickets will breach?" early enough to intervene.
**Metric**: Minimize Financial Loss (not just accuracy).

In [ ]:
# --- 2C.0 GUARDRAILS: Check Data Integrity ---
required_cols = ['Is_SLA_Breach', 'Breach_Cost', 'Ticket Creation Date']
missing = [c for c in required_cols if c not in df_sla.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}. Run SLA + cost attribution cells BEFORE modeling.")

print("✅ Integrity Check Passed: Ready for Modeling")

In [ ]:
# --- 2C.1 Feature Engineering (High Fidelity Time Features) ---
# Using 'Ticket Creation Date' (Timestamp) for precise Interaction Hour

time_col = 'Ticket Creation Date'

# Add Feature to MAIN DATASET (so it persists for dashboard export)
df_sla['Interaction_Hour'] = df_sla[time_col].dt.hour.fillna(12).astype(int)
df_sla['DayOfWeek'] = df_sla[time_col].dt.dayofweek.fillna(2).astype(int)
df_sla['Is_Weekend'] = (df_sla['DayOfWeek'] >= 5).astype(int)

df_model = df_sla.copy()

# Handle Categoricals
cat_cols = ['Ticket Type', 'Ticket Priority', 'Ticket Channel', 'Product Purchased']
for c in cat_cols:
    df_model[c] = df_model[c].fillna('Unknown')

# Define Features & Target
target = 'Is_SLA_Breach_Numeric'
features_cat = cat_cols
features_num = ['Interaction_Hour', 'Is_Weekend', 'DayOfWeek']

print(f"Modeling with {len(features_cat) + len(features_num)} Features.")

In [ ]:
# --- 2C.2 Train/Test Split & Pipeline ---
X = df_model[features_cat + features_num].copy()
y = df_model[target].astype(int).copy()

# Stratified Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Preprocessing Pipeline
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), features_cat),
        ("num", "passthrough", features_num),
    ]
)

# Baseline LR Model
model_lr = LogisticRegression(max_iter=2000, class_weight="balanced")

pipe_lr = Pipeline(steps=[
    ("prep", preprocess),
    ("model", model_lr)
])

pipe_lr.fit(X_train, y_train)

proba_test_lr = pipe_lr.predict_proba(X_test)[:, 1]
print("Baseline LR ROC AUC:", roc_auc_score(y_test, proba_test_lr))
print("Baseline LR PR AUC :", average_precision_score(y_test, proba_test_lr))

In [ ]:
# --- 2C.3 Cost-Based Evaluation (Optimize Threshold) ---
# We pick the threshold that minimizes EXPECTED FINANCIAL LOSS

cost_series = df_model.loc[X_test.index, 'Breach_Cost'].fillna(0).values

def expected_cost_at_threshold(y_true, proba, cost, threshold):
    y_pred = (proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Cost Parameters
    FP_COST = 2 # Operational cost of false alarm (escalation)
    
    fn_cost = cost[(y_true == 1) & (y_pred == 0)].sum() # Missed Breach (Expensive)
    fp_cost = FP_COST * ((y_true == 0) & (y_pred == 1)).sum() # False Alarm (Cheap)
    
    total = fn_cost + fp_cost
    return total, tn, fp, fn, tp

thresholds = np.linspace(0.05, 0.95, 19)
results = []
for t in thresholds:
    total, tn, fp, fn, tp = expected_cost_at_threshold(y_test.values, proba_test_lr, cost_series, t)
    results.append((t, total, tn, fp, fn, tp))

best_lr = sorted(results, key=lambda x: x[1])[0]
print(f"Best LR Threshold: {best_lr[0]:.2f}")
print(f"Expected Test Set Cost: ${best_lr[1]:,.0f}")

In [ ]:
# --- 2C.4 Upgrade Model (Random Forest) + Feature Importance ---

# Random Forest
rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced_subsample",
    min_samples_leaf=3
)

pipe_rf = Pipeline(steps=[("prep", preprocess), ("model", rf)])
pipe_rf.fit(X_train, y_train)

proba_test_rf = pipe_rf.predict_proba(X_test)[:, 1]
print("RF ROC AUC:", roc_auc_score(y_test, proba_test_rf))

# Optimize RF Threshold
results_rf = []
for t in thresholds:
    total, tn, fp, fn, tp = expected_cost_at_threshold(y_test.values, proba_test_rf, cost_series, t)
    results_rf.append((t, total, tn, fp, fn, tp))

best_rf = sorted(results_rf, key=lambda x: x[1])[0]
print(f"Best RF Threshold: {best_rf[0]:.2f}")
print(f"Expected Test Set Cost: ${best_rf[1]:,.0f}")

# Feature Importance (Permutation)
perm = permutation_importance(pipe_rf, X_test, y_test, n_repeats=5, random_state=42, scoring="average_precision")
imp = sorted(zip(X_test.columns, perm.importances_mean), key=lambda x: x[1], reverse=True)

print("\n--- Top Risk Drivers (Permutation Importance) ---")
for k, v in imp[:5]:
    print(f"{k:20s} {v:.4f}")

In [ ]:
# --- 2C.5 Export Risk Scores for Dashboard ---
# Use Best Model (RF) and Threshold
final_model = pipe_rf
final_thresh = best_rf[0]

# Predict on Full Dataset (for Dashboard)
# Note: We reuse the pipeline so preprocessing is consistent
df_sla['Predicted_Breach_Prob'] = final_model.predict_proba(df_model[features_cat + features_num])[:, 1]
df_sla['Risk_Bucket'] = pd.cut(
    df_sla['Predicted_Breach_Prob'],
    bins=[0, 0.33, 0.66, 1.0],
    labels=['Low Risk', 'Medium Risk', 'High Risk'],
    include_lowest=True
)

print("✅ Risk Scores Generated & Attached to Application Data.")
display(df_sla[['Ticket Priority', 'Interaction_Hour', 'Predicted_Breach_Prob', 'Risk_Bucket']].head())

## Phase 3: Optimization & Simulation
**Goal**: Answer "What operational change reduces breaches the most?"

In [ ]:
# --- 3A. Scenario-Ready Optimization Table ---
opt_df = df_sla.copy()

# Baseline metrics
baseline = {
    "tickets": len(opt_df),
    "breach_rate": opt_df['Is_SLA_Breach'].mean(),
    "total_cost": opt_df['Est_Breach_Cost'].sum()
}
print("Baseline State:", baseline)

In [ ]:
# --- 3B. Identify High-Risk Segments ---
segment = (
    opt_df.groupby(['Ticket Priority', 'Ticket Channel'])
    .agg(
        Tickets=('Is_SLA_Breach', 'size'),
        Breach_Rate=('Is_SLA_Breach', 'mean'),
        Total_Cost=('Est_Breach_Cost', 'sum'),
        Avg_Cost=('Est_Breach_Cost', 'mean')
    )
    .reset_index()
    .sort_values('Total_Cost', ascending=False)
)

display(segment.head(10))

In [ ]:
# --- 3C. Define Scenario Function ---
def simulate_intervention(df_in, segment_filter, reduction_rate=0.20):
    """
    segment_filter: boolean mask on df
    reduction_rate: percent reduction in breaches for targeted segment
    """
    df_sim = df_in.copy()
    
    # baseline breaches in target segment
    target_mask = segment_filter
    target_breaches = df_sim.loc[target_mask, 'Is_SLA_Breach'].sum()
    
    # expected breaches avoided
    avoided = int(round(target_breaches * reduction_rate))
    
    # simulate: mark some breaches as "prevented"
    breach_idx = df_sim.loc[target_mask & (df_sim['Is_SLA_Breach'] == 1)].index
    
    # only flip as many as we "avoid"
    if len(breach_idx) > 0:
        flip_idx = breach_idx[:avoided]
        df_sim.loc[flip_idx, 'Is_SLA_Breach'] = False
        df_sim.loc[flip_idx, 'Est_Breach_Cost'] = 0
    
    results = {
        "breaches_avoided": avoided,
        "new_breach_rate": df_sim['Is_SLA_Breach'].mean(),
        "new_total_cost": df_sim['Est_Breach_Cost'].sum(),
        "cost_saved": df_in['Est_Breach_Cost'].sum() - df_sim['Est_Breach_Cost'].sum()
    }
    return results

print("✅ Simulation Engine Ready.")

In [ ]:
# --- 3D. Run 3 Key Scenarios ---

# Scenario 1: Reduce breaches for Critical + Chat by 20%
scenario1 = simulate_intervention(
    opt_df,
    segment_filter=(opt_df['Ticket Priority'] == 'Critical') & (opt_df['Ticket Channel'] == 'Chat'),
    reduction_rate=0.20
)

# Scenario 2: Reduce breaches for High + Email by 15%
scenario2 = simulate_intervention(
    opt_df,
    segment_filter=(opt_df['Ticket Priority'] == 'High') & (opt_df['Ticket Channel'] == 'Email'),
    reduction_rate=0.15
)

# Scenario 3: Reduce breaches for Top 2 cost segments by 10%
top2 = segment.head(2)[['Ticket Priority','Ticket Channel']].values.tolist()

mask_top2 = pd.Series([False] * len(opt_df))
for p, c in top2:
    mask_top2 = mask_top2 | ((opt_df['Ticket Priority'] == p) & (opt_df['Ticket Channel'] == c))

scenario3 = simulate_intervention(
    opt_df,
    segment_filter=mask_top2,
    reduction_rate=0.10
)

print("Scenarios Run.")

In [ ]:
# --- 3E. Scenario Comparison Table ---
comparison = pd.DataFrame([
    {"Scenario": "Baseline", **baseline, "breaches_avoided": 0, "cost_saved": 0},
    {"Scenario": "S1: Critical+Chat 20% reduction", **scenario1},
    {"Scenario": "S2: High+Email 15% reduction", **scenario2},
    {"Scenario": "S3: Top2 Segments 10% reduction", **scenario3},
])

display(comparison)

In [ ]:
# --- DASHBOARD EXPORT ---
# Exports the rigorous data for the Streamlit App

print("Exporting Dashboard Data...")
export_df = df_sla.copy()

# Add ID if missing
if "Ticket ID" not in export_df.columns:
    export_df.insert(0, "Ticket ID", range(1, len(export_df) + 1))

# Realistic Date Simulation (Jan-Mar 2023)
np.random.seed(42)
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-03-31')
days_range = (end_date - start_date).days
random_days = np.random.randint(0, days_range, size=len(export_df))
export_df["Ticket_Date"] = [start_date + pd.Timedelta(days=x) for x in random_days]
export_df["Ticket_Date"] = pd.to_datetime(export_df["Ticket_Date"]).dt.date

# Use the 'Predicted_Breach_Prob' from 2C.5 if available, else 0
if 'Predicted_Breach_Prob' in export_df.columns:
    export_df['Pred_Breach_Prob'] = export_df['Predicted_Breach_Prob']
else:
    export_df['Pred_Breach_Prob'] = 0
    export_df['Risk_Bucket'] = 'N/A'

export_df.rename(columns={"Est_Breach_Cost": "Breach_Cost"}, inplace=True)

# Select Columns
final_cols = ['Ticket ID', 'Ticket_Date', 'Ticket Priority', 'Ticket Channel', 
              'Ticket Type', 'Product Purchased', 'Resolution_Hours', 
              'Is_SLA_Breach', 'Breach_Cost', 'Pred_Breach_Prob', 'Risk_Bucket']
              
df_dashboard = export_df[final_cols].copy()
out_path = "../outputs/dashboard/customer_support_sla_dashboard.csv"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
df_dashboard.to_csv(out_path, index=False)
print(f"✅ Dashboard CSV Exported.")